In [3]:
from enum import Enum
import random
from time import sleep
from matplotlib import pyplot as plt
import numpy as np
import copy
import json
import pandas as pd
import csv
from math import floor

from queue import PriorityQueue, SimpleQueue

In [4]:
# Directional Vectors
DY = [-1, 1, 0, 0]
DX = [0, 0, 1, -1]   

# Represents the different types of Cells
class Cell(Enum):
    BLOCKED = 0
    OPEN = 1
    FIRE = 2
    BUTTON = 3

    def __eq__(self, other):
        if isinstance(other, Cell):
            return self.value == other.value
        return self.value == other
    
# Represents the Ship
class Ship: 
    def __init__(self, D, q, seed, init=True):
        self.D = D
        self.q = q
        self.seed = seed
        self.time = 0
        self.board = []
        self.open_cells = []
        self.fire_cells = []
        self.bot_location = ()
        
        if init:
            self.init_board()
            self.init_bot()
            self.init_fire()
            self.init_button()
    
    def init_board(self):
        n = self.D
        random.seed(self.seed)
        
        self.board = [[Cell.BLOCKED for _ in range(n)] for _ in range(n)]
        rand_row = random.randint(0, n - 1)
        rand_col = random.randint(0, n - 1)
        
        self.board[rand_row][rand_col] = Cell.OPEN
        valid_cells = set()
        open_cells = [(rand_row, rand_col)]
        valid_cells.add((rand_row, rand_col))

        while True:
            candidates = self.get_candidates(valid_cells)
            if not candidates:
                break

            rand_cell = random.choice(candidates)
            r, c = rand_cell
            self.board[r][c] = Cell.OPEN
            valid_cells.add((r, c))
            open_cells.append((r, c))

        # Collect all open cells with exactly 1 open neighbor
        dead_ends = []
        for oc in open_cells:
            r, c = oc
            open_neighbors = self.get_neighbor_count(r, c, Cell.OPEN)
            if open_neighbors == 1:
                dead_ends.append(oc)

        # Open approximately half of the dead ends
        num_dead_ends = len(dead_ends)
        while len(dead_ends) > num_dead_ends // 2:

            # Pick a random dead end and open one of it's neighbors
            rand_idx = random.randint(0, len(dead_ends) - 1)
            r, c = dead_ends[rand_idx]
            closed_neighbors = self.get_neighbors(r, c, Cell.BLOCKED)
            if closed_neighbors:
                rand_cell = random.choice(closed_neighbors)
                nr, nc = rand_cell
                self.board[nr][nc] = Cell.OPEN
                open_cells.append((nr, nc))
                dead_ends.pop(rand_idx)

                # Check for newly opened dead ends
                open_neighbors = self.get_neighbors(nr, nc, Cell.OPEN)
                for neighbor in open_neighbors:
                    r, c = neighbor

                    # If this pair is also a dead end, remove it
                    try:
                        idx = dead_ends.index((r, c))
                        dead_ends.pop(idx)
                    except ValueError: # was not a dead end
                        continue
        ### print(len(dead_ends), num_dead_ends)

        self.open_cells = open_cells

    def deepcopy(self, D, q, seed):
        new_ship = Ship(D, q, seed, init=False)
        new_ship.board = [row[:] for row in self.board]
        new_ship.bot_location = copy.deepcopy(self.bot_location)
        new_ship.button_location = copy.deepcopy(self.button_location)
        new_ship.fire_cells = [row[:] for row in self.fire_cells]
        new_ship.initial_fire = copy.deepcopy(self.initial_fire)
        new_ship.open_cells = [row[:] for row in self.open_cells]
        new_ship.time = 0

        return new_ship

    def init_bot(self):
        self.bot_location = random.choice(self.open_cells)
    
    def init_fire(self):
        """Randomly selects a cell to catch on fire"""
        open_cell = random.choice(self.open_cells) # Choose a random cell from the open cells
        r, c = open_cell

        self.board[r][c] = Cell.FIRE
        self.fire_cells.append(open_cell)
        self.open_cells.remove(open_cell)

        self.initial_fire = (r, c)

    def init_button(self):
        open_cell = random.choice(self.open_cells)
        r, c = open_cell

        self.board[r][c] = Cell.BUTTON 
        self.button_location = (r, c)

    # Time Steps
    def time_step(self):
        self.advance_fire()
        self.time += 1
    
    def advance_fire(self):
        """Advances fire within a calculated probability"""
        fire_cells = self.fire_cells[:]
        
        for fc in fire_cells:
            r, c = fc
            potential_fire = self.get_neighbors(r, c, Cell.OPEN)

            for cell in potential_fire:
                nr, nc = cell
                ignite = self.ignition_probability(nr, nc)
                
                if ignite: # Ignites the cell and removes it from open cells
                    self.board[nr][nc] = Cell.FIRE
                    self.open_cells.remove(cell)
                    self.fire_cells.append(cell)

    
    # Helper Methods
    def is_valid(self, r, c):
        """Determine if a row x col is within the bounds of the board"""
        return r >= 0 and c >= 0 and r < self.D and c < self.D
    
    def get_neighbors(self, r, c, target):
        """Get all of the valid neighbors of the desired type"""
        res = []
        for i in range(4):
            nr = DY[i] + r
            nc = DX[i] + c
            if self.is_valid(nr, nc) and self.board[nr][nc] == target:
                res.append((nr, nc))
        
        return res

    def get_neighbor_count(self, r, c, target):
        """Get the number of valid neighbors of the desired type"""
        return len(self.get_neighbors(r, c, target))

    def get_candidates(self, open_cells):
        n = self.D
        candidates = []
        to_remove = []
        for pair in open_cells:
            r, c = pair
            closed_neighbors = self.get_neighbors(r, c,  Cell.BLOCKED)
            valid_found = 0
            for neighbor in closed_neighbors:
                n_r, n_c = neighbor
                open_neighbors = self.get_neighbor_count(n_r, n_c, Cell.OPEN)
                if open_neighbors == 1:
                    valid_found += 1
                    candidates.append((n_r, n_c))

            if valid_found == 0:
                # print("removing {}".format(pair))
                to_remove.append(pair) 

        for pair in to_remove:
            open_cells.remove(pair)
            
        return candidates

    def set_q(self, new_q):
        self.q = new_q

    def ignition_probability(self, r, c):
        """Calculate the probability that any given cell should be ignited, given the formula: 1 - (1 - q)^K"""
        q = self.q # Flammability constant
        K = self.get_neighbor_count(r, c, Cell.FIRE) # Get number of burning neighbors
        probability = 1 - ((1 - q) ** K)
        return random.uniform(0, 1) <= probability

    def simulate_fire(self):
        SIMULATIONS = 25
        deep_copy = [row[:] for row in self.board]
        open_cells_copy = [row[:] for row in self.open_cells]
        seed = self.seed

        keys = [i for i in range(101) if i % 10 == 0]
        cached_results = {0 : [[0 for _ in range(D)] for _ in range(D)]}
        
        random.seed(None)
        button_r, button_c = self.button_location
        bot_r, bot_c = self.bot_location
        fire_r, fire_c = self.fire_cells[0]

        for i in range(SIMULATIONS):
            
            # Simulate fire
            curr = cached_results[0]
            last = 0
            for k in keys:
                steps = last
                while steps < k:
                    self.advance_fire()
                    if self.board[button_r][button_c] == Cell.FIRE:
                        break
                    steps += 1

                for fc in self.fire_cells:
                    a, b = fc
                    curr[a][b] += 1

                cached_results[k] = [row[:] for row in curr]
                last = k
                
            self.board = [row[:] for row in deep_copy]
            self.fire_cells = [(fire_r, fire_c)]
            self.open_cells = [row[:] for row in open_cells_copy]


        for k in cached_results:
            for i in range(D):
                for j in range(D):
                    cached_results[k][i][j] /= SIMULATIONS
        random.seed(seed)
        self.board = deep_copy
        print("END SIM")
        return cached_results

    def has_path(self, src, dst, radius = 0):
        stack = [(src)]
        visited = set()
        while stack:
            curr = stack.pop()
            visited.add(curr)
            if curr == dst:
                return True

            r, c = curr
            for i in range(4):
                nr = DY[i] + r
                nc = DX[i] + c
                if self.is_valid(nr, nc) and (nr, nc) not in visited and self.board[nr][nc] not in (Cell.FIRE, Cell.BLOCKED):
                    
                    # Bot 3 Case
                    if radius >= 1:
                        if self.get_neighbor_count(nr, nc, Cell.FIRE)  > 0:
                            continue
                            
                    stack.append((nr, nc))
        return False
                
    
    def astar(self, start, goal, radius=-1, weights = None):
        open = PriorityQueue() # fringe
        closed = set() # explored
        
        start_r, start_c = start[0], start[1]
        goal_r, goal_c = goal[0], goal[1]

        f = abs(goal_r - start_r) + abs(goal_c - start_c) if not weights else 0
        open.put((f, (start, [start])))

        while open.qsize() != 0:
            curr = open.get()
            #print("curr", curr)
            f, vertex = curr
            coords, path = vertex

            if coords == goal:
                return path # SUCCESS
            else:
                closed.add(coords)
                
                r, c = coords
                for i in range(4):
                    nr = DY[i] + r
                    nc = DX[i] + c
                    if self.is_valid(nr, nc) and self.board[nr][nc] not in (Cell.BLOCKED, Cell.FIRE) and (nr, nc) not in closed:
                        if radius >= -1:
                            if (nr, nc) == self.initial_fire:
                                closed.add((nr, nc))
                                continue
                        if radius >= 0: # self.board[nr][nc] == Cell.OPEN and 
                            if self.board[nr][nc] == Cell.FIRE:
                                closed.add((nr, nc))
                                continue
                            # check for fire
                        
                        if radius == 1: # BUG: if bot is next to button and fire is also next to button, then bot will not enter button cell
                            if self.get_neighbor_count(nr, nc, Cell.FIRE) > 0:
                                if self.board[nr][nc] != Cell.BUTTON: # ignore if not button
                                    closed.add((nr, nc))
                                    continue
                            # check for neighbor fire

                        new_path = path + [(nr, nc)]

                        g = len(new_path) - 1
                        idx = round(g / 10) * 10 if g <= 100 else 100
                        h = (abs(goal_r - nr) + abs(goal_c - nc) if not weights
                             else weights[idx][nr][nc])
                        print(h)
                        f = g + h

                        next_entry = (f, ((nr, nc), new_path))

                        open.put(next_entry)

        return []
        
    def is_path_obstructed(self, bot, path=[]):
        if path == []:
            return True
        for (r, c) in path:
            if not self.is_valid(r, c):
                continue
            if self.board[r][c] == Cell.FIRE:
                return True
                
            if bot == 3:
                for i in range(4):
                    nr, nc = DY[i] + r, DX[i] + c
                    if self.board[nr][nc] == Cell.FIRE:
                        return True
        return False

    # Bot methods
    def is_fail(self):
        r, c = self.button_location
        if self.board[r][c] == Cell.FIRE:
            return True
        
        r, c = self.bot_location
        if self.board[r][c] == Cell.FIRE:
            return True
        
        return False

    def display(self, path = None):
        """Display a grid image of the current board (FOR USE IN NOTEBOOK)"""
        n = self.D
        copy_board = [row[:] for row in self.board]
        if not isinstance(copy_board[0][0], (float, int)):
            for i in range(n):
                for j in range(n):
                    copy_board[i][j] = copy_board[i][j].value

        if path:
            for pair in path:
                r, c = pair
                copy_board[r][c] = 3
                
        image_data = np.array(copy_board)
        plt.imshow(image_data, "Blues")
        # plt.axis("off")
        plt.show()
                      

In [5]:
class Bot:
    def __init__(self, ship: Ship):
        self.ship = ship
        self.bot = ship.bot_location
        self.button = ship.button_location

    def move(self, new_location):
        self.ship.bot_location = new_location
        self.bot = self.ship.bot_location
    
    def bot_1(self):
        path = self.ship.astar(self.bot, self.button)
        if path:
            return path
        return []

    def bot_2(self):
        path = self.ship.astar(self.bot, self.button, radius=0)
        if path:
            return path
        return []

    def bot_3(self):
        path = self.ship.astar(self.bot, self.button, radius=1)
        if path:
            return path
        return []

    def bot_4(self, weights):
        #cached_weights = self.ship.simulate_fire()
        path = self.ship.astar(self.bot, self.button, weights = weights)
        if path:
            return path
        return [] 
        
    def simulate(self, bot_num, step_limit=-1):
        
        bot_query = (
            self.bot_1 if bot_num == 1 else
            self.bot_2 if bot_num == 2 else
            self.bot_3 if bot_num == 3 else
            self.bot_4
        )
        
        cached_results = self.ship.simulate_fire() if bot_num == 4 else None
        success = False
        path = [] 
        curr_path_ind = 1

        bot_path = []
        i = 0
        while step_limit < 0 or i < step_limit:

            
            # Path is blocked, find a new way
            if self.ship.is_path_obstructed(bot_num, path=path[curr_path_ind:]): # TODO: check that path is not obstructed
                
                # Call DFS to check for path given bot criteria
                if self.ship.has_path(self.button, self.bot, radius = bot_num - 2):
                    path = bot_query() if bot_num < 4 else bot_query(cached_results)
                    curr_path_ind = 1
                else:
                    success = False
                    break

            if path == []:
                success = False
                break

            next = path[curr_path_ind]
            curr_path_ind += 1

            i += 1

            if next == () or next == (-1, -1): # no next move found
                success = False
                break
                
            else: # move bot
                self.move(next)
                bot_path += [next]
                if self.bot == self.button:
                    success = True
                    break

            self.ship.time_step()

            if self.ship.is_fail():
                success = False
                break
        
        return success, i, bot_path


In [ ]:
# Generate Ship
D = 50 # ship size
q = .8   # flammability constant
seed = None

ship = Ship(D, q, seed)
test_bot = Bot(ship)
res, steps, path = test_bot.simulate(4)
ship.display(path)
res, steps

In [ ]:
D = 50 # ship size
q = 1   # flammability constant
seed = None

ship = Ship(D, q, seed)
for key, val in ship.simulate_fire().items():
    print(key)
    copy = [row[:] for row in ship.board]
    ship.board = val
    ship.display()
    ship.board = copy
    

In [ ]:
print(bot_path)

In [7]:
# Load Seeds

SEEDS_FILE_PATH = 'seeds.txt'
with open(SEEDS_FILE_PATH, 'r') as seeds_file:
    temp_seeds_dict = json.load(seeds_file)
seeds_file.close()

SEEDS = dict()
for key in temp_seeds_dict:
    SEEDS[int(key)] = temp_seeds_dict[key]

In [8]:
SHIPS = dict()
for seed_id in SEEDS:
    seed = SEEDS[seed_id]

In [13]:

BOT_1_DATA = []

# COLUMN_HEADERS = ["BOT", "RESULT", "STEPS", "SEED", "Q", "SIZE"]

D = 100
BOTS = [4]
#Q_VALS = [.2, .3, .4, .5, .6, .7, .8, .9, 1]
Q_VALS = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]
# SEEDS

for seed_id in SEEDS:
    if seed_id < 31:
        continue
    seed = SEEDS[seed_id]

    ship = Ship(D, 0, seed)

    for b in BOTS:
        for q in Q_VALS:

            # ship = Ship(D, 0, seed)

            ship_copy = ship.deepcopy(D, q, seed)
            # ship_copy = copy.deepcopy(ship)
            # ship_copy.set_q(q)
            # bot = Bot(ship_copy)

            bot = Bot(ship_copy)            
            result, steps, path = bot.simulate(b)
            
            trial = (b, result, steps, seed_id, q)
            print(f"TRIAL {seed_id}, BOT {b}, Q {int(q*10)}: {trial}")
            #BOT_1_DATA.append(trial)

            csv_file_name = f'../data/bot{b}q{int(q*10)}.csv'

            with open(csv_file_name, mode='a', newline='') as file:
                writer = csv.writer(file)

                #for row in BOT_1_DATA:
                writer.writerow(trial)



END SIM
TRIAL 31, BOT 4, Q 0: (4, True, 1, 31, 0)
END SIM
TRIAL 31, BOT 4, Q 1: (4, True, 1, 31, 0.1)
END SIM
TRIAL 31, BOT 4, Q 2: (4, True, 1, 31, 0.2)
END SIM
TRIAL 31, BOT 4, Q 3: (4, True, 1, 31, 0.3)
END SIM
TRIAL 31, BOT 4, Q 4: (4, True, 1, 31, 0.4)
END SIM
TRIAL 31, BOT 4, Q 5: (4, True, 1, 31, 0.5)
END SIM
TRIAL 31, BOT 4, Q 6: (4, True, 1, 31, 0.6)
END SIM
TRIAL 31, BOT 4, Q 7: (4, True, 1, 31, 0.7)
END SIM
TRIAL 31, BOT 4, Q 8: (4, True, 1, 31, 0.8)
END SIM
TRIAL 31, BOT 4, Q 9: (4, True, 1, 31, 0.9)
END SIM
TRIAL 31, BOT 4, Q 10: (4, True, 1, 31, 1)
END SIM
TRIAL 32, BOT 4, Q 0: (4, True, 50, 32, 0)
END SIM
TRIAL 32, BOT 4, Q 1: (4, True, 50, 32, 0.1)
END SIM
TRIAL 32, BOT 4, Q 2: (4, True, 50, 32, 0.2)
END SIM
TRIAL 32, BOT 4, Q 3: (4, True, 50, 32, 0.3)
END SIM
TRIAL 32, BOT 4, Q 4: (4, True, 50, 32, 0.4)
END SIM
TRIAL 32, BOT 4, Q 5: (4, True, 50, 32, 0.5)
END SIM
TRIAL 32, BOT 4, Q 6: (4, True, 50, 32, 0.6)
END SIM
TRIAL 32, BOT 4, Q 7: (4, True, 50, 32, 0.7)
END SIM
T

KeyboardInterrupt: 

In [ ]:
print(BOT_1_DATA)
print(len(BOT_1_DATA))

In [ ]:
round(26 / 10) * 10

In [ ]:
all_attributes = dir(ship)

class_variables = [attr for attr in all_attributes if not callable(getattr(ship, attr))]
class_variables